In [1]:
import numpy as np
import pandas as pd
import re,math
from sklearn import preprocessing
from scipy.stats import pearsonr
import matplotlib.pyplot as plt


#已有的35细胞系
data = pd.read_csv(r'35_CCLE4000.csv')

In [2]:
data.head()

,Name,MT.CO2,MT.CO1,MT.RNR2,FTL,EEF1A1P5,RPS18,TMSB10,FN1,MT.ND2,...,G3BP1,FAS,MIA,TNFRSF1B,PIK3CD,RAVER1,NEUROD6,IGHV2.26,RP11.254B13.1,MYRF
0,A2058,138.664175,99.733470,29.362917,33.507670,28.580983,32.005938,10.723401,4.332792,46.013772,...,0.622143,-0.016467,0.225654,-0.087723,0.010787,0.076835,-0.093931,-0.092728,0.178902,0.060812
1,A2780,132.079201,114.522279,43.443616,7.071069,41.351143,20.272538,3.218207,-0.070932,47.875397,...,0.383265,-0.039013,-0.078075,-0.077774,-0.044276,0.350657,-0.078075,-0.078075,0.128915,-0.067700
2,A375,88.664555,82.985653,17.684115,15.773244,54.772199,30.988268,19.931180,0.679905,34.807456,...,0.712732,-0.026034,1.234437,-0.080171,-0.032848,0.775303,-0.118437,-0.118437,0.332271,-0.083249
3,A427,129.025616,89.746054,13.120489,11.230936,23.942771,11.706606,29.606541,-0.044986,88.208783,...,0.173741,0.025657,-0.061178,-0.061313,-0.031394,0.105856,-0.061409,-0.061201,0.096060,-0.026113
4,CAOV3,122.213951,116.564153,43.291175,26.778444,35.949574,17.530583,22.951898,1.567967,53.569745,...,0.493755,0.062419,-0.074416,-0.059800,-0.042722,0.163952,-0.077403,-0.076815,0.117359,-0.055078


In [3]:
name_index = [] 
for i in data['Name']:
    name_index.append(str(i).strip())

In [4]:
#原始数据
origin = pd.read_csv(r'CCLE_EXP(name).csv')

In [5]:
c_list = []
for i in origin.columns:
    c_list.append(str(i))
#原始数据细胞系名称合集

In [6]:
#name = re.findall()
real_names = []
for i in name_index:
    text = i + '.*? '
    cout = 0
    for j in c_list:
        temp_list = re.findall(text, j, re.S)
        if len(temp_list) != 0:
            if (c_list[cout] != 'NCIH2342_LUNG ') & (c_list[cout] != 'NCIH2347_LUNG '):
                real_names.append(c_list[cout])
                break
        cout +=1
#补全后的35细胞系名称

In [7]:
temp = []
for i in real_names:
    temp.append(origin[i])

In [8]:
df = pd.DataFrame(temp)
df = df.transpose()#转置
#合并35细胞系数据

In [9]:
df.index = origin['Name']
#替换行索引为基因名称

In [10]:
#保存筛选结果1
#df.to_csv('Screened_First.csv')

In [11]:
#复制数据，避免误操作
temp_z_score = df.copy()

In [12]:
#zscore标准化
values = temp_z_score.values            #Dataframe转换为Array
values = values.astype('float32')       #定义数据类型
scaler = preprocessing.StandardScaler() #创建Scaler对象
temp = scaler.fit_transform(values)     #稀疏矩阵  
z_score=pd.DataFrame(temp,columns=temp_z_score.columns,index = origin['Name']) #将Array还原为Dataframe，保持行列名

In [13]:
#分割每行并计算各行方差(每个基因的方差)
temp = []
for i in origin['Name']:
    te = z_score.loc[i].values.var()
    temp.append(te)

In [14]:
#方差Dataframe
temp1 = pd.DataFrame(temp,index = origin['Name'],columns=['variance'])

In [15]:
#按照方差排序
temp2 = temp1.sort_values(by="variance",ascending=False)  # by指定按哪列排序。ascending表示是否升序

In [16]:
#取【0:指定位置】区间内基因构成dataframe，选取基因
temp_index = temp2[0:10000].index 
draft_first = z_score.loc[temp_index[0]]
t = draft_first
for i in temp_index:
    temp_data = z_score.loc[i]
    t = t[~t.index.duplicated(keep='first')]
    temp_data = temp_data[~temp_data.index.duplicated(keep='first')]
    t = pd.concat([t,temp_data],axis=1)
t.dropna(subset=['MT-CO2'],inplace=True)

In [17]:
#固定区间的标准化后的  基因表达-细胞系   dataframe
df2 = t.transpose().copy()
df2 = df2.drop_duplicates()
df2

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
MT-CO2,138.665405,132.080368,88.665337,129.026764,122.215034,130.080261,147.931213,150.274948,88.670563,121.103760,...,178.673752,103.776741,138.319656,132.701187,159.497208,139.794464,134.107101,106.909088,116.005997,74.758606
FTL,33.507969,7.071132,15.773384,11.231035,26.778683,4.013042,3.810183,8.348491,19.012959,12.412223,...,34.435070,34.991974,6.396748,20.167116,0.791694,21.789629,3.167242,4.184994,19.852560,30.605114
MT-ND2,46.014179,47.875824,34.807766,88.209564,53.570221,55.080532,39.533794,28.851393,78.043709,30.601273,...,25.188253,52.729561,37.203083,53.635742,54.151505,71.736412,47.124008,30.010900,29.537886,20.045937
MT-RNR2,29.363178,43.444000,17.684273,13.120605,43.291561,53.964428,57.979183,57.678417,79.852448,22.386780,...,57.014282,33.238365,59.669582,55.456463,15.444898,21.513290,28.767656,55.377277,14.408346,59.660408
EEF1A1P5,28.581238,41.351509,54.772686,23.942984,35.949894,30.014765,9.664021,37.236958,44.284985,29.743744,...,33.773712,52.704514,47.066711,24.994398,16.572939,18.826252,51.085117,35.221874,19.798355,21.329021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RBM19,0.041057,0.074285,0.086052,0.096579,0.004322,0.099290,-0.000843,-0.002058,0.067767,0.052218,...,0.010564,0.048377,0.091402,0.047783,0.076887,0.047245,0.007265,-0.019071,-0.025976,0.010666
AL162759.1,-0.093932,-0.077762,-0.118438,-0.061455,-0.049462,-0.070288,-0.045639,-0.063563,0.011571,-0.102399,...,-0.051431,-0.101020,-0.081873,-0.081531,-0.059981,-0.083017,-0.075402,-0.101159,0.115353,-0.115860
ZHX2,-0.088900,-0.077936,-0.099230,-0.060565,-0.034806,-0.050469,0.050356,-0.054985,-0.065218,-0.084272,...,-0.031353,0.041815,-0.055507,-0.024688,-0.025235,-0.068520,-0.033527,-0.030822,-0.043734,0.053485
CEP164P1,-0.030867,-0.077900,-0.061717,-0.056134,-0.027352,-0.066341,-0.015491,-0.025422,-0.058048,-0.082554,...,-0.014065,-0.056773,-0.055610,-0.019556,-0.055929,-0.080411,0.002438,-0.080898,-0.094630,0.140995


In [18]:
#保存筛选结果2
#各基因按照方差排序从大到小排序的标准化后的数据
#df2.to_csv('Screened_Second.csv')

In [19]:
#计算皮尔森相关性系数
pearson=df2.corr('pearson')
#spearman.to_csv('spearman.csv')

In [20]:
#保存筛选结果3
pearson.to_csv('pearson.csv')

In [21]:
# 筛数据
tptp = pearson.copy()
dpdp = pearson.copy()
gl = 0.88
# 筛高相似
while True:
    drop_status = 0
    count = 0
    drop_h = 0
    drop_l = 0
    for i in tptp.values:
        cot = 0
        for j in i:
            if j < gl:
                drop_h = count
                drop_l = cot
                drop_status = 1
                break
            cot += 1
        if drop_status == 1:
            break
        count += 1
    
    #取出对应行
    count = 0 
    for i in tptp.values:
        if count == drop_h:
            drop_h_list = i
            break
        count += 1
    
    #取出对应列
    drop_l_list = []
    for i in tptp.values:
        cot = 0
        for j in i:
            if cot == drop_l:
                drop_l_list.append(j)
            cot += 1
    
    h=np.array(drop_h_list).copy()
    l=np.array(drop_l_list).copy()
    h[h < gl]=10000
    l[l < gl]=10000
    if  np.sum(h) >= np.sum(l):
        tptp.drop(index=[tptp.columns[drop_h]],columns=[tptp.columns[drop_h]],inplace=True)
    elif np.sum(h) < np.sum(l):
        tptp.drop(index=[tptp.columns[drop_l]],columns=[tptp.columns[drop_l]],inplace=True)

    p = np.array(tptp.values).copy()
    p[p < gl]=35*35
    if  np.sum(p) < 35*35:
        break

In [22]:
# 筛低相似 
while True:
    dpdp[dpdp == 1]=-1
    drop_status = 0
    count = 0
    drop_h = 0
    drop_l = 0
    for i in dpdp.values:
        cot = 0
        for j in i:
            if j > gl:
                drop_h = count
                drop_l = cot
                drop_status = 1
                break
            cot += 1
        if drop_status == 1:
            break
        count += 1
    
    #取出对应行
    count = 0
    drop_h_list = []
    for i in dpdp.values:
        if count == drop_h:
            drop_h_list = i
            break
        count += 1
    
    #取出对应列
    drop_l_list = []
    for i in dpdp.values:
        cot = 0
        for j in i:
            if cot == drop_l:
                drop_l_list.append(j)
            cot += 1
    
    h=np.array(drop_h_list).copy()
    l=np.array(drop_l_list).copy()
    h[h > gl]=10000
    l[l > gl]=10000
    if  np.sum(h) >= np.sum(l):
        dpdp.drop(index=[dpdp.columns[drop_h]],columns=[dpdp.columns[drop_h]],inplace=True)
    elif np.sum(h) < np.sum(l):
        dpdp.drop(index=[dpdp.columns[drop_l]],columns=[dpdp.columns[drop_l]],inplace=True)
    p = np.array(dpdp.values).copy()
    p[p > gl]=35*35
    if  np.sum(p) < 35*35:
        break
dpdp[dpdp == -1]=1

In [23]:
tptp.to_csv(r'tptp.csv')
dpdp.to_csv(r'dpdp.csv')
high_similar_regiments = pd.read_csv(r'tptp.csv')
low_similar_regiments = pd.read_csv(r'dpdp.csv')

In [24]:
#读取药物对数据  高相似 低相似团
drugs_data = pd.read_csv(r'Label_synergy4000.csv')

In [25]:
#取各团的细胞系名
high_cells = high_similar_regiments.columns[1:]

In [26]:
low_cells = low_similar_regiments.columns[1:]

In [27]:
drugs_temp = []
j = 0
for i in drugs_data.columns[2:]:
    drugs_temp.append(drugs_data[i].rename(real_names[j]))
    j += 1

In [28]:
#预处理drugs
t = drugs_temp[0].rename(real_names[0])
for i in drugs_temp[1:]:
    t = pd.concat([t,i],axis=1)
drugs = t.copy()
drugs

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMEL30_SKIN,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE
0,2.60,15.20,6.22,-16.4000,5.360,-7.080,8.050,12.400,5.79,4.70,...,-6.230,13.50,5.10,5.15,6.63,3.880,-1.48,-8.660,13.00,4.83
1,-4.03,11.10,10.50,-15.6000,16.300,-1.720,8.890,0.686,9.68,-20.30,...,2.820,1.51,13.80,-5.77,-14.40,-10.700,-16.80,-11.800,15.30,-13.10
2,14.50,29.50,17.50,17.7000,10.300,0.830,5.770,19.300,8.40,14.50,...,-12.200,-7.40,10.50,-4.97,13.10,18.800,12.00,5.950,6.77,18.90
3,-8.16,-5.42,-7.75,-14.3000,4.810,0.312,3.410,2.920,-3.48,-2.54,...,-3.140,-13.90,-4.57,-19.60,-2.72,3.680,14.40,-13.600,-3.64,-5.73
4,22.50,22.80,18.00,20.1000,22.700,6.300,24.900,35.400,23.40,37.30,...,24.400,31.20,18.40,12.90,14.30,34.700,21.20,14.400,26.40,55.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,11.70,20.20,9.17,46.6000,0.586,-37.700,-2.100,34.100,18.30,16.50,...,-0.217,-10.90,22.00,-2.43,-43.90,9.340,-4.83,-15.400,-4.42,17.80
579,8.01,7.50,-4.18,-6.7800,3.210,7.240,5.340,9.690,4.67,4.53,...,11.100,7.12,-2.42,6.96,14.90,9.120,10.60,14.700,11.60,14.20
580,12.40,19.20,-5.01,37.4000,8.590,9.770,3.770,10.600,18.20,15.40,...,4.330,5.84,-1.58,18.60,-20.70,25.100,20.30,3.750,-4.29,-1.44
581,4.79,-23.30,-2.70,0.0856,-17.300,19.200,17.400,1.590,-11.60,-16.90,...,-13.900,-2.42,-2.61,19.10,-11.10,-13.500,9.05,-0.306,9.48,-4.29


In [29]:
#zscore标准化
values = drugs.values            #Dataframe转换为Array
values = values.astype('float32')       #定义数据类型
scaler = preprocessing.StandardScaler() #创建Scaler对象
temp = scaler.fit_transform(values)     #稀疏矩阵  
drugs=pd.DataFrame(temp,columns=drugs.columns) #将Array还原为Dataframe，保持行列名

In [30]:
#提取药物对对应高相似团
high_list = []
for i in high_cells:
    high_list.append(drugs[i])
r = high_list[0]
for i in high_list[1:]:
    r = pd.concat([r,i],axis=1)
high_result = r
high_result.to_csv('high_result.csv')

In [31]:
high_result

,A2058_SKIN,A2780_OVARY,CAOV3_OVARY,COLO320_LARGE_INTESTINE,HT29_LARGE_INTESTINE,KPL1_BREAST,MDAMB436_BREAST,NCIH1650_LUNG,NCIH460_LUNG,OVCAR4_OVARY,SKMEL30_SKIN,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE
0,-0.289857,0.266041,-0.021771,-0.517577,-0.282331,-0.834680,-0.470318,-0.201483,-0.031924,0.161841,-0.603652,0.061116,0.105251
1,-0.622916,0.088321,0.721469,-0.263454,0.134112,-0.270218,0.024511,-0.324864,0.053983,-1.108628,-0.105114,-0.337035,-0.456149
2,0.307940,0.885891,0.313842,-0.142556,0.213651,-0.223563,-0.582421,-0.324864,0.314568,-0.410576,-0.932522,-0.307867,0.277968
3,-0.830387,-0.627758,-0.059137,-0.167115,-0.507462,-1.105392,-0.234727,0.100931,-0.433968,-0.095782,-0.433433,-0.841287,-0.144349
4,0.709821,0.595472,1.156272,0.116783,1.349922,1.014223,-0.443460,0.664808,0.718330,0.502985,1.083665,0.343686,0.310003
...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,0.167282,0.482771,-0.346107,-1.969303,-0.099959,-1.099632,-0.763420,0.027691,-0.492958,1.649937,-0.272414,-0.215256,-1.243656
579,-0.018085,-0.067725,-0.167838,0.161349,-1.049881,0.587996,-1.887369,0.330367,-0.602346,0.250068,0.351007,0.127109,0.326020
580,0.202447,0.439425,0.197668,0.281299,0.884051,0.109931,0.042319,0.261589,-0.059987,-0.005790,-0.021933,0.551512,-0.624328
581,-0.179842,-1.402787,-1.561244,0.728385,1.446505,0.760790,3.060924,1.055952,-1.721427,0.063615,-1.026170,0.569742,-0.368055


In [32]:
#提取药物对对应低相似团
low_list = []
for i in low_cells:
    low_list.append(drugs[i])
r = low_list[0]
for i in low_list[1:]:
    r = pd.concat([r,i],axis=1)
low_result = r
low_result

KeyError: 'PA1_OVARY '

In [ ]:
#计算高相似细胞团的斯皮尔曼相关性系数
spearman_h=high_result.corr('spearman')

#计算低相似细胞团的斯皮尔曼相关性系数
spearman_l=low_result.corr('spearman')

In [ ]:
#保存筛选结果4
#spearman_h.to_csv('spearman_h.csv')
#spearman_l.to_csv('Kendall_l.csv')

In [ ]:
#取上三角
arr1 = np.triu(np.array(spearman_h),k=1)
arr2 = np.triu(np.array(spearman_l),k=1)

In [ ]:
#取非零元素
high_res = arr1.ravel()[np.flatnonzero(arr1)]
low_res = arr2.ravel()[np.flatnonzero(arr2)]
result_compare = pd.DataFrame([high_res.T,low_res.T],index=['high','low']).T
result_compare_lst = []
for index, line in result_compare.iteritems():
    result_compare_lst.append(np.array(line.dropna()))

In [ ]:
fig = plt.subplot(1,1,1)
bplot = fig.boxplot(result_compare_lst,
                meanline='black',
                notch=True,
                vert=True, 
                patch_artist=True)
#颜色填充
colors = ['red', 'yellow']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

for median in bplot['medians']:
    median.set(color='black', linewidth=3)
    
fig.yaxis.grid(True) #在y轴上添加网格线
fig.set_xticks([y+1 for y in range(len(result_compare_lst))] ) 
#fig.set_xlabel('细胞团') #设置x轴名称
#fig.set_ylabel('ylabel') #设置y轴名称

# 添加刻度
# 添加刻度名称，我们需要使用 plt.setp() 函数：

# 加刻度名称
plt.setp(fig, xticks=[1,2],
         xticklabels=['high','low'])
# 我们的刻度数是哪些，以及我们想要它添加的刻度标签是什么。
        
plt.show()